In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

%matplotlib inline

C:\Users\amina sultana\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!pip install tensorflow-recommenders
!pip install pprintpp

In [5]:
from typing import Dict, Text
import pprintpp

import numpy as np
import pandas as pd

import tensorflow as tf

import tensorflow_recommenders as tfrs

In [6]:
interaction_data = pd.read_csv("RAW_interactions.csv")
recipe_data = pd.read_csv("RAW_recipes.csv")

interaction_train = pd.read_csv("interactions_train.csv")
interaction_test = pd.read_csv("interactions_test.csv")

In [7]:
from_year, to_year = '2011-01-01','2018-12-31'

recipe_data['submitted'] = pd.to_datetime(recipe_data['submitted'])
recipe_data['submitted'] = recipe_data['submitted'].apply(lambda x: x.tz_localize(None))
recipes_l0y = recipe_data.loc[recipe_data['submitted'].between(from_year, to_year, inclusive=False)]

interaction_data['date'] = pd.to_datetime(interaction_data['date'])
interaction_data['date'] = interaction_data['date'].apply(lambda x: x.tz_localize(None))
interactions_l0y = interaction_data.loc[interaction_data['date'].between(from_year, to_year, inclusive=False)]

print(recipes_l0y.shape)
print(interactions_l0y.shape)

C:\Users\amina sultana\AppData\Local\Temp\ipykernel_23492\1666361360.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  recipes_l0y = recipe_data.loc[recipe_data['submitted'].between(from_year, to_year, inclusive=False)]


(18577, 12)
(297540, 5)


C:\Users\amina sultana\AppData\Local\Temp\ipykernel_23492\1666361360.py:9: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  interactions_l0y = interaction_data.loc[interaction_data['date'].between(from_year, to_year, inclusive=False)]


In [8]:
# Filter the interaction_train and interaction_test dataframes within the same datasets
from_year, to_year = '2011-01-01', '2018-12-31'
interaction_train_filtered = interaction_train.loc[(interaction_train['date'] >= from_year) & (interaction_train['date'] <= to_year)]
interaction_test_filtered = interaction_test.loc[(interaction_test['date'] >= from_year) & (interaction_test['date'] <= to_year)]

# Replace the original dataframes with the filtered dataframes
interaction_train = interaction_train_filtered
interaction_test = interaction_test_filtered

In [9]:
interaction_data = interaction_data.astype({'user_id': 'string', 'recipe_id':'string'})
interaction_train = interaction_train.astype({'user_id': 'string', 'recipe_id':'string'})
interaction_test = interaction_test.astype({'user_id': 'string', 'recipe_id':'string'})

In [10]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueUserIds, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueUserIds)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueFoodIds, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueFoodIds)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])
        
    def call(self, userId, foodId):
        user_embeddings  = self.user_embeddings (userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))

# Build a model.
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])
            

    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userID"], features["foodID"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)

In [11]:
uniqueUserIds = interaction_data.user_id.unique()
uniqueFoodIds = interaction_data.recipe_id.unique()

In [12]:
train_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interaction_train.user_id.values, tf.string),
    "foodID":tf.cast(interaction_train.recipe_id.values, tf.string),
    "rating":tf.cast(interaction_train.rating.values, tf.float32)
})

test_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interaction_test.user_id.values, tf.string),
    "foodID":tf.cast(interaction_test.recipe_id.values, tf.string),
    "rating":tf.cast(interaction_test.rating.values, tf.float32)
})

In [13]:
tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [14]:
model = FoodModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))
cached_train = train_data.shuffle(100_000).batch(8192).cache()
cached_test = test_data.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
17/17 [==============================] - 3s 45ms/step - root_mean_squared_error: 4.6000 - loss: 21.1651 - regularization_loss: 0.0000e+00 - total_loss: 21.1651
Epoch 2/10
17/17 [==============================] - 0s 24ms/step - root_mean_squared_error: 4.5235 - loss: 20.4726 - regularization_loss: 0.0000e+00 - total_loss: 20.4726
Epoch 3/10
17/17 [==============================] - 0s 27ms/step - root_mean_squared_error: 4.4573 - loss: 19.8770 - regularization_loss: 0.0000e+00 - total_loss: 19.8770
Epoch 4/10
17/17 [==============================] - 0s 25ms/step - root_mean_squared_error: 4.3922 - loss: 19.2979 - regularization_loss: 0.0000e+00 - total_loss: 19.2979
Epoch 5/10
17/17 [==============================] - 0s 24ms/step - root_mean_squared_error: 4.3252 - loss: 18.7098 - regularization_loss: 0.0000e+00 - total_loss: 18.7098
Epoch 6/10
17/17 [==============================] - 0s 26ms/step - root_mean_squared_error: 4.2552 - loss: 18.1058 - regularization_loss: 0.0000e

In [15]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 251ms/step - root_mean_squared_error: 3.6987 - loss: 13.6805 - regularization_loss: 0.0000e+00 - total_loss: 13.6805


{'root_mean_squared_error': 3.6987173557281494,
 'loss': 13.680509567260742,
 'regularization_loss': 0,
 'total_loss': 13.680509567260742}

In [19]:
user_rand = uniqueUserIds[200]
test_rating = {}
for m in test_data.take(10):
    test_rating[m["foodID"].numpy()]=RankingModel()(tf.convert_to_tensor([user_rand]),tf.convert_to_tensor([m["foodID"]]))

In [21]:
print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(recipe_data.loc[recipe_data['id'] == int(m.decode())]['name'].item())

Top 10 recommended products for User 450004: 
bread pot fondue
blueberry pancake syrup   low carb
murgh makhani  indian butter chicken
big kahuna personal pizza
easy marinated flank steak
corn capsicum curry
mushroom and sun dried tomato risotto
fantabulously low carb  and yet    phony  rice pudding two ways
sweet and sour pork ribs
two bean toss


In [22]:
print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(recipe_data.loc[recipe_data['id'] == int(m.decode())]['name'].item())

Top 10 recommended products for User 450004: 
bread pot fondue
blueberry pancake syrup   low carb
murgh makhani  indian butter chicken
big kahuna personal pizza
easy marinated flank steak
corn capsicum curry
mushroom and sun dried tomato risotto
fantabulously low carb  and yet    phony  rice pudding two ways
sweet and sour pork ribs
two bean toss
